In [1]:
%tensorflow_version 2.x
!pip install -q sklearn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf
tf.__version__

'2.4.1'

In [2]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", names=["x1", "x2", "x3", "x4", "y"])

In [3]:
df.head()

,x1,x2,x3,x4,y
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
df.describe()

,x1,x2,x3,x4
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [5]:
X = df[["x1", "x2", "x3", "x4"]]

scaler = StandardScaler()
X = scaler.fit_transform(X)

y = df["y"]
y = pd.get_dummies(y)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [7]:
def customized_nn (nodes, extra_layers):
  model = Sequential()
  model.add(Dense(nodes, input_dim=4, activation='relu'))

  for _ in range(extra_layers):
    model.add(Dense(nodes, activation='relu'))
  
  
  model.add(Dense(3, activation='softmax'))

  model.compile(loss='categorical_crossentropy', 
                      optimizer='adam', 
                      metrics=['accuracy'])
  
  model.fit(X_train, y_train,
          batch_size=5,
          epochs=50,
          verbose=0,
          validation_data=(X_test, y_test) 
          )
  
  return model


In [8]:
def find_best_model(nodes, layers):
  high_acc = 0
  low_loss = 1000
  best_model = None

  for n in nodes:
    for l in layers:
      model = customized_nn (n, l)

      test_res = model.evaluate(X_test, y_test, verbose=0)
      print("nodes: " + str(n) + " extra layers: " + str(l) + " --- accuracy " + str(test_res[1]) + " loss " + str(test_res[0]))

      if(test_res[1] - test_res[0] > high_acc - low_loss):
        best_model = model
        high_acc = test_res[1]
        low_loss = test_res[0]
        print("^current best^")
    
  return best_model 
      

In [9]:
nodes = [2, 4, 10, 20]
layers = range(4)

model = find_best_model(nodes, layers)

nodes: 2 extra layers: 0 --- accuracy 0.5666666626930237 loss 0.6904841065406799
^current best^
nodes: 2 extra layers: 1 --- accuracy 0.4333333373069763 loss 1.0257046222686768
nodes: 2 extra layers: 2 --- accuracy 0.6333333253860474 loss 0.5761631727218628
^current best^
nodes: 2 extra layers: 3 --- accuracy 0.8333333134651184 loss 0.4215429127216339
^current best^
nodes: 4 extra layers: 0 --- accuracy 0.8333333134651184 loss 0.36227113008499146
^current best^
nodes: 4 extra layers: 1 --- accuracy 0.9333333373069763 loss 0.2797718048095703
^current best^
nodes: 4 extra layers: 2 --- accuracy 0.7666666507720947 loss 0.4791930615901947
nodes: 4 extra layers: 3 --- accuracy 0.7666666507720947 loss 0.5072798728942871
nodes: 10 extra layers: 0 --- accuracy 0.9666666388511658 loss 0.21364307403564453
^current best^
nodes: 10 extra layers: 1 --- accuracy 1.0 loss 0.07674063742160797
^current best^
nodes: 10 extra layers: 2 --- accuracy 1.0 loss 0.03206599876284599
^current best^
nodes: 10 ex

In [10]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_47 (Dense)             (None, 20)                100       
_________________________________________________________________
dense_48 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_49 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_50 (Dense)             (None, 3)                 63        
Total params: 1,003
Trainable params: 1,003
Non-trainable params: 0
_________________________________________________________________


In [11]:
test_res = model.evaluate(X_test, y_test, verbose=0)
print("The neural network model scores: \n - accuracy " + str(test_res[1]) + " \n - loss " + str(test_res[0]))

The neural network model scores: 
 - accuracy 1.0 
 - loss 0.014876995235681534
